In [1]:
%load_ext yamlmagic

In [2]:
%%yaml model
id: https://example.org/
name: example
prefixes:
  example: https://example.org/
  linkml: https://w3id.org/linkml/
default_prefix: example
default_range: string
imports:
  - linkml:types

classes:
  Thing:
    attributes:
      type:
        designates_type: true
  Vehicle:
    is_a: Thing
    attributes:
      num_wheels:
        range: integer
  Building:
    is_a: Thing
    attributes:
      num_floors:
        range: integer
        
  Container:
    attributes:
      things:
        range: Thing
        multivalued: true
    

In [35]:
model_str = yaml.dump(model)

In [36]:
from linkml_runtime.utils.schemaview import SchemaView

In [37]:
sv = SchemaView(model_str)
schema = sv.schema

In [38]:
from linkml.generators.pydanticgen import PydanticGenerator
import yaml

In [39]:
gen = PydanticGenerator(schema, log_level=ERROR)

In [40]:
print(gen.serialize())

In [41]:
mod = gen.compile_module()

In [42]:
mod.Container.parse_obj({"things": [{"type": "Vehicle", "num_wheels": 2}]})

In [43]:
import pytest

In [44]:
with pytest.raises(ValueError):
  mod.Container.parse_obj({"things": [{"type": "Vehicle", "num_floors": 4}]})

In [45]:
mod.Container(things=[mod.Vehicle(num_wheels=2)])

In [47]:
with pytest.raises(ValueError):
    mod.Container(things=[mod.Thing(type="Vehicle", num_wheels=2)])

## Change type designator range

What happens if the range is a curie?

In [48]:
schema.classes["Thing"].attributes["type"].range = "uriorcurie"
gen = PydanticGenerator(schema, log_level=ERROR)
mod = gen.compile_module()
print(gen.serialize())

In [49]:
mod.Container.parse_obj({"things": [{"type": "example:Vehicle", "num_wheels": 2}]})

In [50]:
schema.classes["Thing"].attributes["type"].range = "uri"
gen = PydanticGenerator(schema, log_level=ERROR)
mod = gen.compile_module()
print(gen.serialize())

## Data Classes

In [29]:
from linkml.generators.pythongen import PythonGenerator
from logging import ERROR

# Note: Jupyter appears to generate output even if the log_level is set.
gen = PythonGenerator(model, log_level=ERROR)
print(gen.serialize())

## Any Of

In [56]:
%%yaml model
id: https://example.org/
name: example
prefixes:
  example: https://example.org/
  linkml: https://w3id.org/linkml/
default_prefix: example
imports:
  - linkml:types

classes:
  Thing:
    attributes:
      type:
        designates_type: true
        range: string
  Vehicle:
    is_a: Thing
    attributes:
      num_wheels:
        range: integer
  Car:
    is_a: Vehicle
    slot_usage:
      num_wheels:
        equals_expression: "2"
  Bicycle:
    is_a: Vehicle
    slot_usage:
      num_wheels:
        equals_expression: "4"
  Building:
    is_a: Thing
    attributes:
      num_floors:
        range: integer
        
  Container:
    attributes:
      things:
        any_of:
          - range: Vehicle
          - range: Building
        multivalued: true
    

In [57]:
model_str = yaml.dump(model)
sv = SchemaView(model_str)
schema = sv.schema

In [58]:
gen = PydanticGenerator(schema, log_level=ERROR)
mod = gen.compile_module()
print(gen.serialize())

In [59]:
# TODO: flatten unions

In [63]:
mod.Container.parse_obj({"things": [{"type": "Vehicle"}]})

In [64]:
# TODO: expressions

## Union Of

In [56]:
%%yaml model
id: https://example.org/
name: example
prefixes:
  example: https://example.org/
  linkml: https://w3id.org/linkml/
default_prefix: example
imports:
  - linkml:types

classes:
  Thing:
    attributes:
      type:
        designates_type: true
        range: string
  Vehicle:
    is_a: Thing
    attributes:
      num_wheels:
        range: integer
  Car:
    is_a: Vehicle
    slot_usage:
      num_wheels:
        equals_expression: "2"
  Bicycle:
    is_a: Vehicle
    slot_usage:
      num_wheels:
        equals_expression: "4"
  Building:
    is_a: Thing
    attributes:
      num_floors:
        range: integer
        
  Container:
    attributes:
      things:
        any_of:
          - range: Vehicle
          - range: Building
        multivalued: true
    